In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.image import ImageDataGenerator


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir("../input"))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

['aptos2019-blindness-detection', 'densenet121weights']


In [2]:
import pandas as pd
import numpy as np
from prettytable import PrettyTable
import pickle
import os

# Image Libs.
from PIL import Image
import cv2

from keras.layers import GlobalAveragePooling2D, Flatten, Dense, GaussianDropout, BatchNormalization, Dropout, Conv2D, MaxPooling2D
from keras.models import Model
from keras import regularizers, optimizers
from tensorflow.keras.optimizers import Adam
from keras.applications.densenet import DenseNet121

from keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau,CSVLogger)


In [3]:
train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
train_df['diagnosis'] = train_df['diagnosis'].astype('str')
train_df['id_code'] = train_df['id_code'].astype(str)+'.png'


In [4]:

IMG_SIZE=224

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

def circle_crop(img, sigmaX = 30):   
    """
    Create circular crop around image centre    
    """    
    img = crop_image_from_gray(img)    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img=cv2.addWeighted(img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
    return img 




def preprocess_image(img):
    img = circle_crop(img) 
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    return img

In [5]:
datagen=ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    zca_whitening = True)

batch_size = 16
image_size = 224

train_gen=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../input/aptos2019-blindness-detection/train_images",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical",
    target_size=(image_size,image_size),
    subset='training')

test_gen=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../input/aptos2019-blindness-detection/train_images",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical", 
    target_size=(image_size,image_size),
    subset='validation')


/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:337: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 2930 validated image filenames belonging to 5 classes.
Found 732 validated image filenames belonging to 5 classes.


In [6]:
y_train = train_df['diagnosis']
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
num_classes = y_train.shape[1]

In [7]:

def create_model():
    dense_121 = DenseNet121(weights='../input/densenet121weights/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False)
    x = dense_121.get_layer('conv5_block16_2_conv').output
    x = Conv2D(32, (3, 3), input_shape=[96,96,3], activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(32, activation='relu')(x)
    # and a logistic layer -- let's say we have 5 classes
    predictions = Dense(5, activation='softmax')(x)
    model = Model(inputs=dense_121.input, outputs=predictions)
    return model


model=create_model()
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, amsgrad=True), metrics=['accuracy'])

2022-04-20 12:16:29.350193: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-20 12:16:29.454690: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-20 12:16:29.455575: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-20 12:16:29.457618: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [8]:
# warm up model
for layer in model.layers:
    layer.trainable = False

for i in range(-5,0):
    model.layers[i].trainable = True

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(1e-3))

model.fit_generator(
    train_gen,
    steps_per_epoch=(len(train_gen)),
    epochs=2,
    workers=2, 
    use_multiprocessing=True,
    verbose=1)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:739: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
2022-04-20 12:16:37.514091: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Pas

Epoch 1/2


2022-04-20 12:16:46.701195: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


184/184 [==============================] - 288s 1s/step - loss: 0.9862
Epoch 2/2
184/184 [==============================] - 249s 1s/step - loss: 0.7958


In [9]:
for layer in model.layers:
    layer.trainable = True

In [10]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early= EarlyStopping(monitor='val_loss', mode ='min', verbose = 1, patience = 20)
checkpoint = ModelCheckpoint('../working/dr-densenet121.h5', monitor='val_loss', save_best_only = False, mode ='min', verbose = 1)
csv_logger = CSVLogger(filename='../working/training_log.csv',
                       separator=',',
                       append=True)
callbacks_list=[early,checkpoint,csv_logger]

In [11]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer=Adam(learning_rate=1e-4, amsgrad=True), 
    metrics=['accuracy'])

model.fit_generator(train_gen,              
                                    steps_per_epoch=len(train_gen),
                                    validation_data=test_gen,                    
                                    validation_steps=len(test_gen),
                                    epochs=25,
                                    callbacks = callbacks_list, 
                                    use_multiprocessing = True,
                                    verbose=1)

Epoch 1/25
184/184 [==============================] - 463s 2s/step - loss: 0.7353 - accuracy: 0.7259 - val_loss: 0.7143 - val_accuracy: 0.7309

Epoch 00001: saving model to ../working/dr-densenet121.h5
Epoch 2/25
184/184 [==============================] - 436s 2s/step - loss: 0.5706 - accuracy: 0.7645 - val_loss: 0.5818 - val_accuracy: 0.7582

Epoch 00002: saving model to ../working/dr-densenet121.h5
Epoch 3/25
184/184 [==============================] - 428s 2s/step - loss: 0.5112 - accuracy: 0.7863 - val_loss: 0.6117 - val_accuracy: 0.7637

Epoch 00003: saving model to ../working/dr-densenet121.h5
Epoch 4/25
184/184 [==============================] - 427s 2s/step - loss: 0.4881 - accuracy: 0.8078 - val_loss: 0.5616 - val_accuracy: 0.8046

Epoch 00004: saving model to ../working/dr-densenet121.h5
Epoch 5/25
184/184 [==============================] - 420s 2s/step - loss: 0.4351 - accuracy: 0.8276 - val_loss: 0.5788 - val_accuracy: 0.7650

Epoch 00005: saving model to ../working/dr-dense